Nuclear Physics Approach Anomalous Phenomena in E<18 Ke V Hydrogen Ion Beam Implantation Experiments on Pd and Ti WANG Tieshan* PIAO Yubo** HAO Jifang* WANG Xuezhi** JIN Genming* and NIU Zhanqi** *Institute of Modern Physics, Chinese Academy of Sciences Lanzhou 730000, P.R. China **Department of Modern Physics of Lanzhou University Lanzhou 730000, P.R. China


In [143]:
# Imports
import os
import re
import copy
import numpy as np
import pdfplumber
import nltk
import openai
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pdf_dir = "./data"
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
title_abstract_list = []
print(pdf_files)

# testDir="./data/doc16.pdf"

# print(extract_title_abstract(testDir)[0])

In [ ]:
def extract_title_abstract(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]

        text = page.extract_text(x_tolerance=1, y_tolerance=5)

        # print(page.chars)
        table={}
        for x,i in enumerate(page.chars):
            # print(i["text"])
            # if i["text"]==" " or i["text"]=="\n":
            #     print("new line")
            if i["size"] not in table:
                table[i["size"]]=""
            if x>0 and i["x0"]-page.chars[x-1]["x1"]>=1:
                table[i["size"]]+=" "
            if x>0 and page.chars[x-1]["y1"]-i["y1"]>5:
                table[i["size"]]+=" "
            table[i["size"]]+=i["text"]


        # print(text,"\n") 
        tmp=copy.deepcopy(table)

        for i in tmp:
            if len(tmp[i])<30:
                del table[i]

        title=table[max(table.keys())].strip().replace("  "," ")
        abstract_match = re.search(r"Abstract\s+(.*?[.!?])\s*\n[c,©]", text, re.DOTALL)
        if not abstract_match:
            abstract_match = re.search(r'(?i)abstract\b\.?:?]?\s+((?:.|\n)+?)(?=\n\d+\.|Key|Introduction|[0,1]?\Z)', text, re.DOTALL)

        if abstract_match:
            abstract = abstract_match.group(1).strip()

    if not abstract_match:
        abstract = "No abstract found"
    # abstract = text.split('\n')
    # print("Title: ",title,"Abstract: ",abstract)
    # print(title, "\n")

    return title, abstract

# 1B

In [ ]:


def get_text_from_pdf(pdf_path):
    allText=""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
             allText+=page.extract_text(x_tolerance=1, y_tolerance=5)
    return allText

lemmatizer=WordNetLemmatizer()


def preprocess_text(text):
    words=word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered= [lemmatizer.lemmatize(w.lower()) for w in words if not w in stop_words and w.isalpha()]
    return filtered


In [ ]:
embeddings={}

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

# embs=get_embedding("\n".join(filtered))

for i in pdf_files:
    text=get_text_from_pdf("./data/"+i)
    filtered="\n".join(preprocess_text(text))
    if len(filtered)>8192:
        filtered=filtered[:8192]
    embeddings[i]=get_embedding(filtered)
    

In [ ]:
embeddings["doc16.pdf"]

In [ ]:
avg_embedding=[np.mean(np.array(embeddings[i]), axis=0) for i in embeddings]

In [ ]:
avg_embedding=np.array(avg_embedding)
avg_embedding

In [163]:
# Word2Vec 

def docVector(filteredSentence):
    modelW2V = Word2Vec(sentences= filteredSentence, vector_size=len(filteredSentence), window=5, min_count=1, workers=4)
    allVecs = [modelW2V.wv[word] for word in modelW2V.wv.key_to_index]
    return np.mean(allVecs, axis = 0)  # Represents all vectors' means for document


resultsW2V = {}

text=get_text_from_pdf("./data/doc3.pdff")
filtered="\n".join(preprocess_text(text))
resultsW2V[10] = docVector(filtered)

resultsW2V

FileNotFoundError: [Errno 2] No such file or directory: './data/doc3.pdff'

In [ ]:
# Word2Vec Similarity Docs


In [130]:
import random

random_indices = random.sample(range(len(pdf_files)), 5)
random_indices

[14, 20, 29, 23, 18]

In [157]:
random_embeddings = [avg_embedding[i] for i in random_indices]
random_documents = [pdf_files[i] for i in random_indices]

similarity=cosine_similarity(np.array(random_embeddings).reshape(-1,1), avg_embedding.reshape(-1,1))

In [159]:
for i, (document, similarities) in enumerate(zip(random_documents, similarity)):
    # Exclude self-similarity by setting it to a negative value
    similarities[random_indices[i]] = -1
    most_similar_indices = np.argsort(similarities)[-3:]
    print(f'Most similar documents to Document {i + 1}:')
    for j, idx in enumerate(reversed(most_similar_indices), 1):
        print(f'{j}. Document {idx + 1} (Similarity: {similarities[idx]:.4f})')


Most similar documents to Document 1:
1. Document 32 (Similarity: 1.0000)
2. Document 31 (Similarity: 1.0000)
3. Document 2 (Similarity: 1.0000)
Most similar documents to Document 2:
1. Document 32 (Similarity: 1.0000)
2. Document 31 (Similarity: 1.0000)
3. Document 2 (Similarity: 1.0000)
Most similar documents to Document 3:
1. Document 32 (Similarity: 1.0000)
2. Document 31 (Similarity: 1.0000)
3. Document 2 (Similarity: 1.0000)
Most similar documents to Document 4:
1. Document 32 (Similarity: 1.0000)
2. Document 31 (Similarity: 1.0000)
3. Document 2 (Similarity: 1.0000)
Most similar documents to Document 5:
1. Document 32 (Similarity: 1.0000)
2. Document 31 (Similarity: 1.0000)
3. Document 2 (Similarity: 1.0000)


# Word2Vec

w2VModel = Word2Vec(filtered, min_count = 1, vector_size = 100, window = 5, workers = 4)


all_vectors = [w2V.wv[word] for word in model.wv.key_to_index]
mean_vector = np.mean(all_vectors, axis=0)

print("Mean vector:", mean_vector)